In [3]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
from collections import Counter
import requests
import os

In [4]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re

## LDA Classifier

In [5]:
# returns a dictionary of priors (one key, value pair for each category)
def compute_priors(df, response):
    y = df[response]
    priors_dict = {}
    priors = dict(y.value_counts(normalize=True))
    return priors

# returns a dictionary of mu vectors (one key, value pair for each category)
def compute_mu_vectors(df, response):
    y = df[response]
    mu_vectors = {}
    for category_k in y.unique():
        df_k = df[y == category_k] # df with y = category_k
        X_k = df_k.drop(response, axis=1)
        mu_vectors[category_k] = dict(X_k.mean())
    return mu_vectors

# returns the inverse of the covariance matrix
def compute_inv_sigma(df, response):
    X = df.drop(response, axis=1)
    return inv(X.cov())

# returns the classification of a single obs
def classify_obs(x_i, y, mu_vectors, priors, inv_sigma):
        prob_dict = {}
        for category_k in y.unique():
            mu_k = pd.Series(mu_vectors[category_k]).to_numpy()
            first_term = x_i.transpose().dot(inv_sigma).dot(mu_k)
            second_term = .5 * mu_k.transpose().dot(inv_sigma).dot(mu_k)
            third_term = np.log(priors[category_k])
            prob_k = first_term - second_term + third_term
            prob_dict[category_k] = prob_k

        best_class, max_prob = next(iter(prob_dict.items()))
        for class_k, prob_k in prob_dict.items():
            if max_prob < prob_k:
                max_prob = prob_k
                best_class = class_k
        return best_class

class LDA:
    
    def fit(self, X_train, y_train):
        df_train = X_train.copy()
        response = y_train.name
        df_train[response] = y_train
        self.y = df_train[response]
        self.priors = compute_priors(df_train, response)
        self.mu_vectors = compute_mu_vectors(df_train, response)
        self.inv_sigma = compute_inv_sigma(df_train, response)
        
    def predict(self, df_test):
        y_pred = {}
        for i in range(len(df_test)):
            x_i = df_test.loc[i, :].to_numpy()
            y_pred[i] = classify_obs(x_i, self.y, self.mu_vectors, self.priors, self.inv_sigma)
        return pd.Series(y_pred)
    

## Function to Compute Confusion Matrix

In [6]:
def produce_confusion_matrix(y_pred, y_test):
    res = pd.DataFrame({"Prediction": y_pred, "Actual": y_test})

    res_positives = res[res["Actual"] == 1]
    res_negatives = res[res["Actual"] == -1]

    positives_dict = dict(res_positives["Prediction"].value_counts(normalize=True))
    TPs = positives_dict[1]
    FNs = positives_dict[-1]

    negatives_dict = dict(res_negatives["Prediction"].value_counts(normalize=True))
    TNs = negatives_dict[-1]
    FPs = negatives_dict[1]

    positives = pd.Series([TPs, FNs])
    negatives = pd.Series([FPs, TNs])

    confusion_matrix = pd.DataFrame({"Actually Positive": positives, "Actually Negative": negatives})
    confusion_matrix.rename({0: "Prediced Positive", 1: "Predicted Negative"}, inplace=True)
    return confusion_matrix


In [7]:
df_train = pd.read_csv("tf_idf_train2.csv").drop("Unnamed: 0", axis=1)
df_test = pd.read_csv("tf_idf_test2.csv").drop("Unnamed: 0", axis=1)

In [8]:
df_train.head()

,life,comedy,whole,reality,cartoon,good,also,stories,behaved,cross,...,amazons-dr,now-over,creature-basically,mud-wrestling,multi-sexual,barabarian,gamorrean,d-grade,highly-entertaining,Label
0,1.734095,2.38033,2.250752,3.455865,4.257334,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000000,2.38033,0.000000,0.000000,0.000000,2.900542,2.707422,3.329807,7.082109,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.675593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [9]:
df_test.head()

,good,well,comedy,overcome,coaxed,also,love,something,must,tv,...,aggrandizement,uneventfully,feng,satans,day-for-night,preppie,dines,bleibtreu,terminating,Label
0,0.997067,2.532548,2.375586,5.330841,8.740337,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.997067,1.266274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000000,3.798821,0.000000,0.000000,0.000000,1.379756,1.735455,1.821889,2.212013,2.49423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.997067,3.798821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
X_train = df_train.drop("Label", axis=1)
y_train = df_train["Label"]
X_test = df_test.drop("Label", axis=1)
y_test = df_test["Label"]

model = LDA()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
(y_pred == y_test).mean()

In [ ]:
confusion_matrix = produce_confusion_matrix(y_pred, y_test)
confusion_matrix

# Modeling

In [3]:
training_df = pd.read_csv("Training_data.csv").drop("Unnamed: 0",axis=1)
training_df.head()

,File,Review,Label
0,2893_10.txt,Walt Disney's CINDERELLA takes a story everybo...,1
1,7944_9.txt,"Have you ever, or do you have, a pet who's bee...",1
2,11725_10.txt,"I suck at gratuitous Boob references, so i'm j...",1
3,1587_10.txt,"Does anyone know, where I can see or download ...",1
4,10297_8.txt,Well not actually. This movie is very entertai...,1


In [4]:
testing_df = pd.read_csv("Test_data.csv").drop("Unnamed: 0",axis=1)
testing_df.head()

,File,Review,Label
0,2893_10.txt,"""Rush in Rio"" is, no doubt, one of the most ex...",1
1,8705_10.txt,I have seen a number of horror movies to know ...,1
2,11725_10.txt,I'm a fan of B grade 80s films in which the he...,1
3,9859_8.txt,"I think that Pierre Léaud, or his character, t...",1
4,12409_10.txt,This picture doesn't have any big explosions o...,1


In [5]:
# Stop words
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

In [6]:
# Positive semantic words
url = "https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/positive-words.txt"
r = requests.get(url)
s=';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n; \n; Opinion Lexicon: Positive\n;\n; This file contains a list of POSITIVE opinion words (or sentiment words).\n;\n; This file and the papers can all be downloaded from \n;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html\n;\n; If you use this list, please cite one of the following two papers:\n;\n;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." \n;       Proceedings of the ACM SIGKDD International Conference on Knowledge \n;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, \n;       Washington, USA, \n;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing \n;       and Comparing Opinions on the Web." Proceedings of the 14th \n;       International World Wide Web conference (WWW-2005), May 10-14, \n;       2005, Chiba, Japan.\n;\n; Notes: \n;    1. The appearance of an opinion word in a sentence does not necessarily  \n;       mean that the sentence expresses a positive or negative opinion. \n;       See the paper below:\n;\n;       Bing Liu. "Sentiment Analysis and Subjectivity." An chapter in \n;          Handbook of Natural Language Processing, Second Edition, \n;          (editors: N. Indurkhya and F. J. Damerau), 2010.\n;\n;    2. You will notice many misspelled words in the list. They are not \n;       mistakes. They are included as these misspelled words appear \n;       frequently in social media content. \n;\n;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;'
pos_words = r.text[len(s)+2:]
pos_words = pos_words.split("\n")

In [7]:
# Negative semantic words
url = "https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/negative-words.txt"
r = requests.get(url)
s=';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n; \n; Opinion Lexicon: Negative\n;\n; This file contains a list of NEGATIVE opinion words (or sentiment words).\n;\n; This file and the papers can all be downloaded from \n;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html\n;\n; If you use this list, please cite one of the following two papers:\n;\n;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." \n;       Proceedings of the ACM SIGKDD International Conference on Knowledge \n;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, \n;       Washington, USA, \n;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing \n;       and Comparing Opinions on the Web." Proceedings of the 14th \n;       International World Wide Web conference (WWW-2005), May 10-14, \n;       2005, Chiba, Japan.\n;\n; Notes: \n;    1. The appearance of an opinion word in a sentence does not necessarily  \n;       mean that the sentence expresses a positive or negative opinion. \n;       See the paper below:\n;\n;       Bing Liu. "Sentiment Analysis and Subjectivity." An chapter in \n;          Handbook of Natural Language Processing, Second Edition, \n;          (editors: N. Indurkhya and F. J. Damerau), 2010.\n;\n;    2. You will notice many misspelled words in the list. They are not \n;       mistakes. They are included as these misspelled words appear \n;       frequently in social media content. \n;\n;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n\n'
neg_words = r.text[len(s):]
neg_words = neg_words.split("\n")

## Engineering the raw data

In [8]:
# Vectorizing reviews in training set
words_train = (
    training_df.Review.
    str.lower().
    str.replace("[^\w\s]","").
    str.split()
)
bag_of_words_train = words_train.apply(Counter)

In [9]:
# Vectorizing reviews in test set
words_test = (
    testing_df.Review.
    str.lower().
    str.replace("[^\w\s]","").
    str.split()
)
bag_of_words_test = words_test.apply(Counter)

In [10]:
# Removing stop words in training
reviews_train = []
for r in words_train:
    good = []
    for w in r:
        if w not in en_stops:
            good.append(w)
    reviews_train.append(good)
reviews_train = pd.Series(reviews_train)

In [11]:
# Removing stop words in test
reviews_test = []
for r in words_test:
    good = []
    for w in r:
        if w not in en_stops:
            good.append(w)
    reviews_test.append(good)
reviews_test = pd.Series(reviews_test)

In [12]:
# Counting positive and negative words in training set
pos_set = set(pos_words)
neg_set = set(neg_words)
posc_train = []
negc_train = []
for r in reviews_train:
    count_pos = len(pos_set.intersection(set(r)))
    count_neg = len(neg_set.intersection(set(r)))
    posc_train.append(count_pos)
    negc_train.append(count_neg)
    

In [13]:
# Counting positive and negative words in test set
pos_set = set(pos_words)
neg_set = set(neg_words)
posc_test = []
negc_test = []
for r in reviews_test:
    count_pos = len(pos_set.intersection(set(r)))
    count_neg = len(neg_set.intersection(set(r)))
    posc_test.append(count_pos)
    negc_test.append(count_neg)

In [14]:
y_train = training_df.Label
X_train = pd.DataFrame({"Positive_counts":posc_train, "Negative_counts":negc_train})

In [15]:
y_test = testing_df.Label
X_test = pd.DataFrame({"Positive_counts":posc_test, "Negative_counts":negc_test})

## LDA classifier results

In [40]:
model = LDA()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
(y_pred == y_test).mean()

0.73068

In [41]:
confusion_matrix = produce_confusion_matrix(y_pred, y_test)
confusion_matrix

,Actually Positive,Actually Negative
Prediced Positive,0.7208,0.25944
Predicted Negative,0.2792,0.74056


## Adding Interactions

In [17]:
X_train2 = X_train.copy()
X_test2 = X_test.copy()
y_train2 = y_train.copy()
y_test2 = y_test.copy()

In [18]:
X_train2["Interaction_posc_negc"] = X_train2.Positive_counts*X_train2.Negative_counts
X_test2["Interaction_posc_negc"] = X_test2.Positive_counts*X_test2.Negative_counts

In [19]:
model2 = LDA()
model2.fit(X_train2, y_train2)
y_pred2 = model2.predict(X_test2)
(y_pred2 == y_test2).mean()

0.73484

# Squaring Terms

In [20]:
X_train3 = X_train2.copy()
X_test3 = X_test2.copy()
y_train3 = y_train2.copy()
y_test3 = y_test2.copy()

In [21]:
X_train3["Positive_counts2"] = X_train3.Positive_counts**2
X_train3["Negative_counts2"] = X_train3.Negative_counts**2
X_test3["Positive_counts2"] = X_test3.Positive_counts**2
X_test3["Negative_counts2"] = X_test3.Negative_counts**2

In [22]:
model3 = LDA()
model3.fit(X_train3, y_train3)
y_pred3 = model3.predict(X_test3)
(y_pred3 == y_test3).mean()

0.73428

# Cubing Terms

In [27]:
X_train4 = X_train3.copy()
X_test4 = X_test3.copy()
y_train4 = y_train3.copy()
y_test4 = y_test3.copy()

In [28]:
X_train4["Positive_counts3"] = X_train4.Positive_counts**3
X_train4["Negative_counts3"] = X_train4.Negative_counts**3
X_test4["Positive_counts3"] = X_test4.Positive_counts**3
X_test4["Negative_counts3"] = X_test4.Negative_counts**3

In [33]:
model4 = LDA()
model4.fit(X_train4, y_train4)
y_pred4 = model4.predict(X_test4)
(y_pred4 == y_test4).mean()

0.73424